In [10]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import os.path

import string

import re

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load('en')
from spacy.symbols import nsubj

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from settings import *
from utils import *

In [22]:
from gloveEmbeddings import *
loadGloveEmbeddings(gloveFile)

In [19]:
def claimExtraction(limitDocuments=10):
    query = createQuery(limitDocuments, 'web')
    documents = queryDB(query)

    
    source = ['paper', 'report', 'study', 'analysis', 'research', 'survey']
    people = ['expert', 'scientist']
    subject = source + people
    predicate = ['prove', 'demonstrate', 'reveal', 'state', 'mention', 'report', 'say', 'show', 'announce', 'claim', 'suggest']


    
    subjectVec = [word2vec(s) for s in subject]
    predicateVec = [word2vec(p) for p in predicate]
    
    
    def findPhrase(title, body, subjectVec, predicateVec):
        
        for s in sent_tokenize(body):
            subjectFound = predicateFound = False
            claim = ""
            for w in wordpunct_tokenize(s):
                
                if predicateFound == True:
                    claim = s
                    break
                
                wVec = word2vec(w)
                
                if subjectFound == False:
                    for sVec in subjectVec:
                        if abs(cosine_similarity(sVec, wVec)) > 0.9:
                            subjectFound = True
                            break
                        
                if subjectFound == True:
                    for pVec in predicateVec:
                        if abs(cosine_similarity(pVec, wVec)) > 0.9:
                            predicateFound = True
                            break
    
    def findPhrase2(title, body, subjectVec, predicateVec):
        
        for s in sent_tokenize(body):
            subjectFound = predicateFound = False
            claim = ""
            holder = ""
            
            try:
                doc = nlp(s)
                rootVerb = [w for w in doc if w.head is w][0]
            except:
                continue
            
            wVec = word2vec(str(rootVerb))
            for pVec in predicateVec:
                if abs(cosine_similarity(pVec, wVec)) > 0.9:
                    predicateFound = True
                    break

                        
            if predicateFound == True:
                for np in doc.noun_chunks:
                    if(np.root.dep == nsubj and np.root.head == rootVerb):
                        wVec = word2vec(str(np.root.text))
                    
                        for sVec in subjectVec:    
                            if abs(cosine_similarity(sVec, wVec)) > 0.5:
                                    subjectFound = True
                                    holder = np.text
                                    break
                
            if subjectFound == True:
                    claim = s
                    print('claim:', claim)
                    print('by:', holder)
                    break
   


        

#    for s in source:
#        pairs.append((findEmbedding(s),findEmbedding(a)))
    
    documents.apply(lambda d: findPhrase2(d['title'],d['body'], subjectVec, predicateVec), axis=1)

claimExtraction(20)    

claim: Critics say that gives food companies an incentive to turn to experts they believe will look kindly upon their ingredients and gives scientists incentive to do so.
by: Critics
claim: Despite industry opposition to the listing of added sugars, officials say they're confident the science is strong enough to justify adding it to the label.
by: officials
claim: On the front lines of medicine, physicians report that they regularly see rising amounts of resistant infections in patients for whom the resistance has no obvious explanation -- for example, in patients who have not been treated in a hospital or other health-care facility where antibiotics might have been overused or misused.
by: physicians
claim: We believe that the careful wording of the NAS ‘Statement of Task’ is designed to emasculate your review, down to the careful choice of language that avoids an evaluation of ‘data.’ We urge you to follow Admiral Gehman’s example and include “root cause and circumstances” and a spec

In [39]:
def argumentExtraction(limitDocuments=10):
    query = createQuery(limitDocuments, 'web')
    documents = queryDB(query)    

 

    
    documents['bodyVecs'] = documents['body'].apply(lambda b: body2Vec(b))
    documents['titleVec'] = documents['title'].apply(lambda t: title2Vec(t))
    
    
    def nn(titleVec, bodyVecs):
        print("1",len(bodyVecs), bodyVecs[1].shape, titleVec.shape)
        nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(bodyVecs)
        print("222")
        distances, indices = nbrs.kneighbors(titleVec)
        print("2",bodyVecs[0].shape, titleVec.shape, indices.shape)
        
        print(indices.shape)
        #return indices.tolist()
    
    documents['nn'] = documents.apply(lambda d: nn(d['titleVec'],d['bodyVecs']), axis=1)
        
        
    #t, b = documents.iat[3,0], documents.iat[3,1]

    #sent_text = 
    return documents
    
    #loadGloveEmbeddings(gloveFile)
    #print(t)
    #print(b)    
    #print (documents['title'].to_string())
argumentExtraction()    

1 38 (1, 50) (1, 50)
1 38 (1, 50) (1, 50)


ValueError: ('Found array with dim 3. Estimator expected <= 2.', 'occurred at index 0')